In [11]:
import numpy as np
import os 
import pandas as pd 

In [129]:
# scan the directory and store the filenames
files = []

for dirname, _, filenames in os.walk('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database'):
    for filename in filenames:
        files.append(dirname+'/'+filename)

ddl_files = [filename for filename in files if filename.endswith(".sql")]

# extract the db_id from the filename
db_id = []

for i in range(len(ddl_files)):
    file_path = ddl_files[i]
    db_id.append(file_path.split('/')[8])
    
# initilalize ddl table    
ddl_tbl = pd.DataFrame({'db_id':db_id, 'CREATE':'', 'INSERT':''})

In [130]:
# append CREATE TABLE and INSERT queries for each db_id
for i in range(len(ddl_files)):
    create_script = ''
    insert_script = ''

    with open(ddl_files[i], 'r') as f:
        for line in f:
            if not any(keyword in line for keyword in ['PRAGMA', 'BEGIN TRANSACTION', 'COMMIT', '--', 'DROP', '***', '<']):
                if ('INSERT' in line) or ('insert' in line):
                    insert_script += line
                else:
                    create_script += line
                    
    ddl_tbl.iloc[i, 1], ddl_tbl.iloc[i, 2] = create_script.replace('\n',' ').replace('\t', ' '), insert_script.replace('\n',' ').replace('\t', ' ')

In [131]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.reset_option('all')

In [132]:
insert_script = ''
with open('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/car_1/car_1.txt') as f:
    for line in f:
        if ('INSERT' in line):
            insert_script += line
            
ddl_tbl.loc[(ddl_tbl['db_id'] == 'car_1'), 'INSERT']= insert_script

In [133]:
insert_script = ''
with open('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/flight_2/flight_2.txt') as f:
    for line in f:
        if ('INSERT' in line):
            insert_script += line
    
ddl_tbl.loc[(ddl_tbl['db_id'] == 'flight_2'), 'INSERT']= insert_script

In [134]:
insert_script = ''
with open('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/wine_1/wine_1.txt') as f:
    for line in f:
        if ('INSERT' in line):
            insert_script += line

ddl_tbl.loc[(ddl_tbl['db_id'] == 'wine_1'), 'INSERT']= insert_script

In [135]:
insert_script = ''
with open('/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/student_1/student_1.txt') as f:
    for line in f:
        if ('INSERT' in line):
            insert_script += line

ddl_tbl.loc[(ddl_tbl['db_id'] == 'student_1'), 'INSERT']= insert_script

In [136]:
spider = pd.read_csv('/Users/kunwooshin/Downloads/SPIDER_248.csv', index_col=False)

In [137]:
db_list1 = set(spider['db_id'].unique())

In [138]:
db_list2 = set(ddl_tbl['db_id'].unique())

In [139]:
db_list1 - db_list2

{'chinook_1', 'epinions_1', 'icfp_1', 'small_bank_1', 'twitter_1', 'voter_1'}

In [140]:
##### SQLITE FILE #####
import sqlite3

dir_name = '/Users/kunwooshin/Desktop/VLDB/nl2sql/spider/database/'
db_id_lite = ['chinook_1', 'epinions_1', 'icfp_1', 'small_bank_1', 'twitter_1', 'voter_1', 'formula_1', '']
file_path_lite = [dir_name + db_id  for db_id in db_id_lite]

files_lite = []
for file_path in file_path_lite:
    files_lite += (os.listdir(file_path))

ddl_files_lite = [filename for filename in files_lite if filename.endswith(".sqlite")]

ddl_tbl_lite = pd.DataFrame({'db_id':db_id_lite, 'CREATE':'', 'INSERT':None})

for i in range(len(ddl_files_lite)):
    create_script = ''
    insert_script_lite = ''
    
    conn = sqlite3.connect(dir_name + db_id_lite[i] + '/' + ddl_files_lite[i])
    cur = conn.cursor()
    
    # extract CREATE TABLE queries
    cur.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    while True:
        line = cur.fetchone()
        if line is None:
            break
        create_script += line[0]
    
    # extract sample rows of tables for each db_ib
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    table_list = cur.fetchall()

    for (table_name,) in table_list:
        rows = []
        
        query = f"SELECT * FROM {table_name}" 
        cur.execute(query)
        
        while True:
            line = cur.fetchone()
            if line is None:
                break
            insert_q = f"INSERT INTO {table_name} VALUES {line}; \n"
            insert_script_lite += insert_q

    conn.close()
    
    ddl_tbl_lite.iloc[i, 1], ddl_tbl_lite.iloc[i, 2] = create_script.replace('\n',' '), insert_script_lite.replace('\n',' ')

In [141]:
ddl_tbl_lite

,db_id,CREATE,INSERT
0,chinook_1,CREATE TABLE `Album` ( `AlbumId` integer NOT...,"INSERT INTO Album VALUES (1, 'For Those About ..."
1,epinions_1,CREATE TABLE `item` ( `i_id` integer NOT NUL...,"INSERT INTO item VALUES (0, 'pear'); INSERT I..."
2,icfp_1,"CREATE TABLE Inst ( instID INTEGER, name T...","INSERT INTO Inst VALUES (1000, 'University of ..."
3,small_bank_1,CREATE TABLE ACCOUNTS ( custid BIGINT...,"INSERT INTO ACCOUNTS VALUES (1, 'Brown'); INS..."
4,twitter_1,CREATE TABLE `follows` ( `f1` int(11) NOT NU...,"INSERT INTO follows VALUES (1, 2); INSERT INT..."
5,voter_1,CREATE TABLE `AREA_CODE_STATE` ( `area_code`...,"INSERT INTO AREA_CODE_STATE VALUES (201, 'NJ')..."
6,formula_1,"CREATE TABLE ""circuits"" ( ""circuitId"" INTEGER ...","INSERT INTO circuits VALUES (1, 'albert_park',..."
7,csu_1,"CREATE TABLE ""Campuses"" ( \t""Id"" INTEGER PRIMA...","INSERT INTO Campuses VALUES (1, 'California St..."


In [142]:
ddl_tbl_concat = pd.concat([ddl_tbl, ddl_tbl_lite], ignore_index=True)

In [143]:
ddl_tbl_concat.shape

(166, 3)

In [145]:
ddl_tbl_concat.to_csv('ddl_tbl.csv')